In [88]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, desc

from config import api_key

import numpy as np
import requests
import json

In [89]:
# Import files as DataFrames
fourteen_fiftenn_pd = pd.read_csv("draft_data/2014-15.csv")
seventeen_eighteen_pd = pd.read_csv("draft_data/2017-18.csv", encoding = "ISO-8859-1")
twenty_twentyone = pd.read_csv("draft_data/2020-21.csv", encoding = "ISO-8859-1")

In [90]:
# Rename columns to be uniform across dataframes
twenty_twentyone.rename(columns={"Facility name":"Facility Name"}, inplace=True)

In [91]:
# Remove rows of corporate total data
fourteen_fiftenn_pd = fourteen_fiftenn_pd[fourteen_fiftenn_pd['Facility Name'] != 'Corporate Total']
seventeen_eighteen_pd = seventeen_eighteen_pd[seventeen_eighteen_pd['Facility Name'] != 'Corporate Total']
twenty_twentyone = twenty_twentyone[twenty_twentyone['Facility Name'] != 'Corporate Total']

In [92]:
# Drop columns of un-needed data
fourteen_fiftenn_pd = fourteen_fiftenn_pd.drop(columns={"Type", "Scope 1 (t CO2-e)","Scope 2 (t CO2-e)","Total Emissions (t CO2-e)", "Emission  Intensity (t/Mwh)", "Grid Connected", "Grid", "Important Notes"})
seventeen_eighteen_pd = seventeen_eighteen_pd.drop(columns={"Type", "Total Scope 1 Emissions \n(t CO2-e)","Total Scope 2 Emissions \n(t CO2-e)","Total Emissions \n(t CO2-e)", "Emission Intensity \n(t CO2-e/ MWh)", "Grid Connected", "Grid"})
twenty_twentyone = twenty_twentyone.drop(columns={"Type", "Total scope 1 emissions \n(t CO2-e)","Total scope 2 emissions \n(t CO2-e)","Total emissions \n(t CO2-e)", "Emission intensity \n(t CO2-e/ MWh)", "Grid connected", "Grid"})


In [93]:
# Add year column for filtering in plots
fourteen_fiftenn_pd['Year'] = '2014-2015'

seventeen_eighteen_pd['Year'] = '2017-2018'

twenty_twentyone['Year'] = '2020-2021'

In [94]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in fourteen_fiftenn_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']
    state = row['State']

    # update address key value
    params['address'] = f"{location}+{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        fourteen_fiftenn_pd.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        fourteen_fiftenn_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        fourteen_fiftenn_pd.loc[index, "Lat"] = "NaN"
        fourteen_fiftenn_pd.loc[index, "Lng"] = "NaN"

# Visualize to confirm lat lng appear
fourteen_fiftenn_pd.head()

,Reporting Entity,Facility Name,State,Electricity Production (GJ),Electricity Production (Mwh),Primary Fuel,Year,Lat,Lng
0,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,NSW,567719.0,157700.0,Wind,2014-2015,-34.690562,149.427009
1,ACCIONA ENERGY OCEANIA PTY LTD,Royalla Solar Farm,ACT,213115.0,59199.0,Solar,2014-2015,-35.489793,149.143002
2,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,VIC,2461803.0,683834.0,Wind,2014-2015,-37.394639,143.638601
4,AGL ENERGY LIMITED,Banimboola Hydro,VIC,137094.0,38082.0,Hydro,2014-2015,-36.534314,147.459344
5,AGL ENERGY LIMITED,Bayswater Power Station,NSW,50275548.0,13965430.0,Black Coal,2014-2015,-32.392578,150.949051


In [95]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in seventeen_eighteen_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']
    state = row['State']

    # update address key value
    params['address'] = f"{location}+{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        seventeen_eighteen_pd.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        seventeen_eighteen_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        seventeen_eighteen_pd.loc[index, "Lat"] = "NaN"
        seventeen_eighteen_pd.loc[index, "Lng"] = "NaN"

seventeen_eighteen_pd.head()

,Reporting Entity,Facility Name,State,Electricity Production (GJ),Electricity Production (MWh),Primary Fuel,Year,Lat,Lng
0,ACCIONA ENERGY OCEANIA PTY. LTD.,Cathedral Rocks Wind Farm,SA,"550,950","153,042",Wind,2017-2018,NaN,NaN
1,ACCIONA ENERGY OCEANIA PTY. LTD.,Gunning Wind Farm,NSW,"601,740","167,150",Wind,2017-2018,-34.690562,149.427009
2,ACCIONA ENERGY OCEANIA PTY. LTD.,Mt Gellibrand Wind Farm,VIC,"4,320","1,200",Wind,2017-2018,-38.232778,143.792222
3,ACCIONA ENERGY OCEANIA PTY. LTD.,Waubra Wind Farm,VIC,"2,317,673","643,798",Wind,2017-2018,-37.394639,143.638601
5,AGL ENERGY LIMITED,Banimboola Hydro,VIC,"11,089","3,080",Hydro,2017-2018,-36.534314,147.459344


In [96]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in twenty_twentyone.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']
    state = row['State']

    # update address key value
    params['address'] = f"{location} + {state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        twenty_twentyone.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        twenty_twentyone.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        twenty_twentyone.loc[index, "Lat"] = "NaN"
        twenty_twentyone.loc[index, "Lng"] = "NaN"

twenty_twentyone.head()

,Reporting entity,Facility Name,State,Electricity production (GJ),Electricity production (MWh),Primary fuel,Year,Lat,Lng
0,ACCIONA ENERGY OCEANIA PTY LTD,Cathedral Rocks Wind Farm,SA,"512,455","142,348",Wind,2020-2021,NaN,NaN
1,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,NSW,"575,522","159,867",Wind,2020-2021,-34.690562,149.427009
2,ACCIONA ENERGY OCEANIA PTY LTD,Mt Gellibrand Wind Farm,VIC,"1,018,498","282,916",Wind,2020-2021,-38.232778,143.792222
3,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,VIC,"2,339,624","649,896",Wind,2020-2021,-37.394639,143.638601
5,ADANI RENEWABLE ASSET HOLDINGS PTY LTD,Adani Rugby Run Solar Farm,QLD,"587,856","163,293",Solar,2020-2021,-22.144063,147.914256


In [98]:
# Drop rows which returned no coordinates for facilities
fourteen_fiftenn_pd = fourteen_fiftenn_pd[fourteen_fiftenn_pd['Lat'] != 'NaN']

seventeen_eighteen_pd = seventeen_eighteen_pd[seventeen_eighteen_pd['Lat'] != 'NaN']

twenty_twentyone = twenty_twentyone[twenty_twentyone['Lat'] != 'NaN']

In [99]:
twenty_twentyone.head()

,Reporting entity,Facility Name,State,Electricity production (GJ),Electricity production (MWh),Primary fuel,Year,Lat,Lng
1,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,NSW,"575,522","159,867",Wind,2020-2021,-34.690562,149.427009
2,ACCIONA ENERGY OCEANIA PTY LTD,Mt Gellibrand Wind Farm,VIC,"1,018,498","282,916",Wind,2020-2021,-38.232778,143.792222
3,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,VIC,"2,339,624","649,896",Wind,2020-2021,-37.394639,143.638601
5,ADANI RENEWABLE ASSET HOLDINGS PTY LTD,Adani Rugby Run Solar Farm,QLD,"587,856","163,293",Solar,2020-2021,-22.144063,147.914256
7,AGL ENERGY LIMITED,Banimboola Hydro,VIC,"7,240","2,011",Hydro,2020-2021,-36.534314,147.459344


In [ ]:
# fourteen_fifteen, seventeen_eighteen, twenty_twentyone
# filter each df to group by state and then group by again to show primary fuel types

In [ ]:
# // Energy Tables: Titled 14_15_Energy_Production, 17_18_Energy_Production & 20_21_Energy_Production
# // Energy Tables Column Headers:
# // # Reporting entity
# // # Facility name
# // # State
# // # Electricity production (GJ)
# // # Electricity production (MWh)

# DELETE THESE COLUMNS
# // # Total scope 1 emissions (t CO2-e)
# // # Total scope 2 emissions (t CO2-e)
# // # Total emissions (t CO2-e)
# // # Emission intensity (t CO2-e/ MWh)
# // # Grid connected
# // # Grid

# // # Primary fuel
# Lat
# Long
# // # Year
# // # Renewable

# // Population Table Titled Aus_Population
# // Aus_Population Column Headers:
# // # State
# // # Male
# // # Female
# // # Total

# // Income Table Titled Aus_Income
# // Aus_Income Column Headers:
# // # State
# // # Median Income

In [129]:
# fourteen_fifteen_cols = ["Reporting_Entity", "Facility_Name", "State", "Electricity_Production_GJ", "Electricity_Production_MWh","Primary_Fuel", "Lat", "Long", "Year"]
# fourteen_fifteen_transformed = fourteen_fiftenn_pd[fourteen_fifteen_cols].copy()

fourteen_fifteen_transformed = fourteen_fiftenn_pd.rename(columns={"Reporting Entity ": "Reporting_Entity",
                                                                   "Facility Name": "Facility_Name",
                                                                   "Electricity Production (GJ)": "Electricity_Production_GJ",
                                                                   "Electricity Production (Mwh)": "Electricity_Production_MWh",
                                                                   "Primary Fuel": "Primary_Fuel"})

In [153]:
# seventeen_eighteen_cols = ["Reporting_Entity", "Facility_Name", "State", "Electricity_Production_GJ", "Electricity_Production_MWh","Primary_Fuel", "Lat", "Long", "Year"]
# seventeen_eighteen_transformed = seventeen_eighteen_pd[seventeen_eighteen_cols].copy()

seventeen_eighteen_transformed = seventeen_eighteen_pd.rename(columns={"Reporting Entity": "Reporting_Entity",
                                                                   "Facility Name": "Facility_Name",
                                                                   "Electricity Production (GJ)": "Electricity_Production_GJ",
                                                                   "Electricity Production (MWh)": "Electricity_Production_MWh",
                                                                   "Primary Fuel": "Primary_Fuel"})

In [119]:
# seventeen_eighteen_cols = ["Reporting_Entity", "Facility_Name", "State", "Electricity_Production_GJ", "Electricity_Production_MWh","Primary_Fuel", "Lat", "Long", "Year"]
# seventeen_eighteen_transformed = seventeen_eighteen_pd[seventeen_eighteen_cols].copy()

twenty_twentyone_transformed = twenty_twentyone.rename(columns={"Reporting entity": "Reporting_Entity",
                                                                   "Facility Name": "Facility_Name",
                                                                   "Electricity production (GJ)": "Electricity_Production_GJ",
                                                                   "Electricity production (MWh)": "Electricity_Production_MWh",
                                                                   "Primary fuel": "Primary_Fuel"})

In [162]:
renewables = ['Hydro', 'Landfill Gas', 'Wind', 'Solar', 'Bagasse', 'Biofuel', 'Macadamia Nut Shells', 'Biogas']

fourteen_fifteen_transformed['Renewable'] = fourteen_fifteen_transformed.Primary_Fuel.isin(renewables)
seventeen_eighteen_transformed['Renewable'] = seventeen_eighteen_transformed.Primary_Fuel.isin(renewables)
twenty_twentyone_transformed['Renewable'] = twenty_twentyone_transformed.Primary_Fuel.isin(renewables)

In [139]:
fourteen_fifteen_transformed.drop(columns=['index'], inplace=True)

In [157]:
twenty_twentyone_transformed.head()

,Reporting_Entity,Facility_Name,State,Electricity_Production_GJ,Electricity_Production_MWh,Primary_Fuel,Year,Lat,Lng,Renewable
0,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,NSW,"575,522","159,867",Wind,2020-2021,-34.690562,149.427009,True
1,ACCIONA ENERGY OCEANIA PTY LTD,Mt Gellibrand Wind Farm,VIC,"1,018,498","282,916",Wind,2020-2021,-38.232778,143.792222,True
2,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,VIC,"2,339,624","649,896",Wind,2020-2021,-37.394639,143.638601,True
3,ADANI RENEWABLE ASSET HOLDINGS PTY LTD,Adani Rugby Run Solar Farm,QLD,"587,856","163,293",Solar,2020-2021,-22.144063,147.914256,True
4,AGL ENERGY LIMITED,Banimboola Hydro,VIC,"7,240","2,011",Hydro,2020-2021,-36.534314,147.459344,True


In [158]:
seventeen_eighteen_transformed.drop(columns=['index'], inplace=True)
twenty_twentyone_transformed.drop(columns=['index'], inplace=True)

KeyError: "['index'] not found in axis"

In [112]:
# import psycopg2

# connection_string = "postgres:postgres@localhost:5432/energy_db"
# engine = create_engine(f'postgresql://{connection_string}')

# # Confirm tables
# engine.table_names()

# fourteen_fifteen_transformed.to_sql(name='Fourteen_Fifteen_Energy_Production', con=engine, if_exists='append', index=False)
# seventeen_eighteen_transformed.to_sql(name='Seventeen_Eighteen_Energy_Production', con=engine, if_exists='append', index=False)
# twenty_twentyone_transformed.to_sql(name='Twenty_Twentyone_Energy_Production', con=engine, if_exists='append', index=False)

In [172]:
fourteen_fifteen_transformed.to_csv('transformed_csvs/14-15.csv')
seventeen_eighteen_transformed.to_csv('transformed_csvs/17-18.csv')
twenty_twentyone_transformed.to_csv('transformed_csvs/20-21.csv')

In [28]:
# Import files as DataFrames
ff_transformed = pd.read_csv("transformed_csvs/14-15.csv")
se_transformed = pd.read_csv("transformed_csvs/17-18.csv")
tt_transformed = pd.read_csv("transformed_csvs/20-21.csv")

In [23]:
import pymongo

# Setup connection to mongodb
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Create database and collection to use
db = client['energy_db']


In [24]:
fourt_fift_energy_production = db.Fourteen_Fifteen_Energy_Production
sevt_eight_energy_production = db.Seventeen_Eighteen_Energy_Production
twe_twentyo_energy_production = db.Twenty_Twentyone_Energy_Production
aus_population = db.Aus_Population
aus_income = db.Aus_Income

In [29]:
fourt_fift_energy_production.insert_many(ff_transformed.to_dict('records'))
sevt_eight_energy_production.insert_many(se_transformed.to_dict('records'))
twe_twentyo_energy_production.insert_many(tt_transformed.to_dict('records'))

visualisation that shows each state renewables growth? Ie, a double bar graph, showing the amount of renewable plants in WA in 2013 next to the amount of renewable plants in WA in 2020 Using Dataframes 14/15, 17/18 & 20/21.


In [59]:
# ff_transformed
# se_transformed
# tt_transformed

In [41]:
ff_transformed_progression = ff_transformed[["State", "Year", "Renewable"]]
ff_transformed_progression.head(2)

,State,Year,Renewable
0,NSW,2014-2015,True
1,ACT,2014-2015,True


In [54]:
ff_transformed_progression_renewable = ff_transformed_progression.loc[ff_transformed_progression["Renewable"] == True]
ff_transformed_progression_renewable

,State,Year,Renewable
0,NSW,2014-2015,True
1,ACT,2014-2015,True
2,VIC,2014-2015,True
3,VIC,2014-2015,True
5,VIC,2014-2015,True
...,...,...,...
290,SA,2014-2015,True
291,SA,2014-2015,True
297,NSW,2014-2015,True
298,QLD,2014-2015,True


In [55]:
# group dataframe by State
ff_transformed_progression_renewable_grouped = ff_transformed_progression_renewable.groupby(["State"], as_index=False).count()
ff_transformed_progression_renewable_grouped

,State,Year,Renewable
0,ACT,4,4
1,NSW,38,38
2,NT,2,2
3,QLD,14,14
4,SA,10,10
5,TAS,26,26
6,VIC,37,37
7,WA,15,15


In [58]:
# format DF
ff_transformed_progression_renewable_grouped = ff_transformed_progression_renewable_grouped[["State", "Renewable"]]
ff_transformed_progression_renewable_grouped.rename(columns={'Renewable': 'Renewable_Energy_Plants_2014_2015'}, inplace=True)
ff_transformed_progression_renewable_grouped

,State,Renewable_Energy_Plants_2014_2015
0,ACT,4
1,NSW,38
2,NT,2
3,QLD,14
4,SA,10
5,TAS,26
6,VIC,37
7,WA,15


In [63]:
ff_transformed_progression_nonrenewable = ff_transformed_progression.loc[ff_transformed_progression["Renewable"] == False]
# ff_transformed_progression_nonrenewable

# group dataframe by State
ff_transformed_progression_nonrenewable_grouped = ff_transformed_progression_nonrenewable.groupby(["State"], as_index=False).count()
# ff_transformed_progression_nonrenewable_grouped

# format DF
ff_transformed_progression_nonrenewable_grouped = ff_transformed_progression_nonrenewable_grouped[["State", "Renewable"]]
ff_transformed_progression_nonrenewable_grouped.rename(columns={'Renewable': 'Non_Renewable_Energy_Plants_2014_2015'}, inplace=True)
ff_transformed_progression_nonrenewable_grouped

,State,Non_Renewable_Energy_Plants_2014_2015
0,NSW,32
1,NT,13
2,QLD,30
3,SA,13
4,TAS,2
5,VIC,17
6,WA,63


In [71]:
energy_progression_merged = pd.merge(ff_transformed_progression_renewable_grouped, ff_transformed_progression_nonrenewable_grouped, how="left", on=["State", "State"])
# energy_progression_merged

In [42]:
se_transformed_progression = se_transformed[["State", "Year", "Renewable"]]
se_transformed_progression.head(2)

,State,Year,Renewable
0,NSW,2017-2018,True
1,VIC,2017-2018,True


In [61]:
se_transformed_progression_renewable = se_transformed_progression.loc[se_transformed_progression["Renewable"] == True]
# se_transformed_progression_renewable

# group dataframe by State
se_transformed_progression_renewable_grouped = se_transformed_progression_renewable.groupby(["State"], as_index=False).count()
# se_transformed_progression_renewable_grouped

# format DF
se_transformed_progression_renewable_grouped = se_transformed_progression_renewable_grouped[["State", "Renewable"]]
se_transformed_progression_renewable_grouped.rename(columns={'Renewable': 'Renewable_Energy_Plants_2017_2018'}, inplace=True)
se_transformed_progression_renewable_grouped

,State,Renewable_Energy_Plants_2017_2018
0,ACT,3
1,NSW,45
2,NT,2
3,QLD,14
4,SA,12
5,TAS,28
6,VIC,40
7,WA,20


In [72]:
energy_progression_merged = pd.merge(energy_progression_merged, se_transformed_progression_renewable_grouped, how="left", on=["State", "State"])
# energy_progression_merged

In [64]:
se_transformed_progression_nonrenewable = se_transformed_progression.loc[se_transformed_progression["Renewable"] == False]
# se_transformed_progression_nonrenewable

# group dataframe by State
se_transformed_progression_nonrenewable_grouped = se_transformed_progression_nonrenewable.groupby(["State"], as_index=False).count()
# se_transformed_progression_nonrenewable_grouped

# format DF
se_transformed_progression_nonrenewable_grouped = se_transformed_progression_nonrenewable_grouped[["State", "Renewable"]]
se_transformed_progression_nonrenewable_grouped.rename(columns={'Renewable': 'Non_Renewable_Energy_Plants_2017_2018'}, inplace=True)
se_transformed_progression_nonrenewable_grouped

,State,Non_Renewable_Energy_Plants_2017_2018
0,NSW,28
1,NT,64
2,QLD,32
3,SA,12
4,VIC,14
5,WA,71


In [73]:
energy_progression_merged = pd.merge(energy_progression_merged, se_transformed_progression_nonrenewable_grouped, how="left", on=["State", "State"])
# energy_progression_merged

In [43]:
tt_transformed_progression = tt_transformed[["State", "Year", "Renewable"]]
tt_transformed_progression.head(2)

,State,Year,Renewable
0,NSW,2020-2021,True
1,VIC,2020-2021,True


In [62]:
tt_transformed_progression_renewable = tt_transformed_progression.loc[tt_transformed_progression["Renewable"] == True]
# tt_transformed_progression_renewable

# group dataframe by State
tt_transformed_progression_renewable_grouped = tt_transformed_progression_renewable.groupby(["State"], as_index=False).count()
# tt_transformed_progression_renewable_grouped

# format DF
tt_transformed_progression_renewable_grouped = tt_transformed_progression_renewable_grouped[["State", "Renewable"]]
tt_transformed_progression_renewable_grouped.rename(columns={'Renewable': 'Renewable_Energy_Plants_2020_2021'}, inplace=True)
tt_transformed_progression_renewable_grouped

,State,Renewable_Energy_Plants_2020_2021
0,ACT,5
1,NSW,64
2,NT,3
3,QLD,46
4,SA,16
5,TAS,30
6,VIC,55
7,WA,22


In [74]:
energy_progression_merged = pd.merge(energy_progression_merged, tt_transformed_progression_renewable_grouped, how="left", on=["State", "State"])
# energy_progression_merged

In [65]:
tt_transformed_progression_nonrenewable = tt_transformed_progression.loc[tt_transformed_progression["Renewable"] == False]
# tt_transformed_progression_nonrenewable

# group dataframe by State
tt_transformed_progression_nonrenewable_grouped = tt_transformed_progression_nonrenewable.groupby(["State"], as_index=False).count()
# tt_transformed_progression_nonrenewable_grouped

# format DF
tt_transformed_progression_nonrenewable_grouped = tt_transformed_progression_nonrenewable_grouped[["State", "Renewable"]]
tt_transformed_progression_nonrenewable_grouped.rename(columns={'Renewable': 'Non_Renewable_Energy_Plants_2020_2021'}, inplace=True)
tt_transformed_progression_nonrenewable_grouped

,State,Non_Renewable_Energy_Plants_2020_2021
0,NSW,22
1,NT,63
2,QLD,65
3,SA,15
4,VIC,14
5,WA,71


In [75]:
energy_progression_merged = pd.merge(energy_progression_merged, tt_transformed_progression_nonrenewable_grouped, how="left", on=["State", "State"])
energy_progression_merged

,State,Renewable_Energy_Plants_2014_2015,Non_Renewable_Energy_Plants_2014_2015,Renewable_Energy_Plants_2017_2018,Non_Renewable_Energy_Plants_2017_2018,Renewable_Energy_Plants_2020_2021,Non_Renewable_Energy_Plants_2020_2021
0,ACT,4,NaN,3,NaN,5,NaN
1,NSW,38,32.0,45,28.0,64,22.0
2,NT,2,13.0,2,64.0,3,63.0
3,QLD,14,30.0,14,32.0,46,65.0
4,SA,10,13.0,12,12.0,16,15.0
5,TAS,26,2.0,28,NaN,30,NaN
6,VIC,37,17.0,40,14.0,55,14.0
7,WA,15,63.0,20,71.0,22,71.0


In [78]:
# import pymongo

# # Setup connection to mongodb
# conn = 'mongodb://localhost:27017'
# client = pymongo.MongoClient(conn)

# # Create database and collection to use
# db = client['energy_db']

# energy_progression_merged = db.energy_progression_merged

# energy_progression_merged.insert_many(energy_progression_merged.to_dict('records'))

Which state produces the most energy per capita? which state produces the least? Divided into renewable vs non renewable.

In [132]:
# Read in CSV
population_summary = pd.read_csv("state_population_summary.csv")

# format DF
population_summary.rename(columns={"Unnamed: 0": "State", "Total": "Population_Total"}, inplace=True)
population_summary = population_summary[["State", "Population_Total"]]

# Removing the Australia(b) total line in index 8
population_summary = population_summary.drop(8)

population_summary

,State,Population_Total
0,New South Wales,8072163
1,Victoria,6503491
2,Queensland,5156138
3,South Australia,1781516
4,Western Australia,2660026
5,Tasmania,557571
6,Northern Territory,232605
7,Australian Capital Territory,454499


In [133]:
population_summary.at[0, 'State'] = "NSW"
population_summary.at[1, 'State'] = "VIC"
population_summary.at[2, 'State'] = "QLD"
population_summary.at[3, 'State'] = "SA"
population_summary.at[4, 'State'] = "WA"
population_summary.at[5, 'State'] = "TAS"
population_summary.at[6, 'State'] = "NT"
population_summary.at[7, 'State'] = "ACT"
population_summary

,State,Population_Total
0,NSW,8072163
1,VIC,6503491
2,QLD,5156138
3,SA,1781516
4,WA,2660026
5,TAS,557571
6,NT,232605
7,ACT,454499


In [134]:
# population_summary["State"].replace("New South Wales", "NSW", regex=True)
# population_summary

In [135]:
# # you can do multiple replacements in within one call of the replace method by creating a mapping dictionary
# # very scalable woo
# replacement_mapping_dict = {
#     "New South Wales": "NSW",
#     "Victoria": "VIC",
#     "Queensland": "QLD",
#     "South Australia": "SA",
#     "Western Australia": "WA",
#     "Tasmania": "TAS",
#     "Northern Territory": "NT",
#     "Australian Capital Territory": "ACT",
# }
# population_summary["State"].replace(replacement_mapping_dict, regex=True)

# population_summary

In [ ]:
# energy_per_capita = state_population_summary.csv

In [148]:
tt_eppc = tt_transformed[["State", "Electricity_Production_MWh", "Renewable"]]
# tt_eppc

tt_eppc_renewable = tt_eppc.loc[tt_eppc["Renewable"] == True]
# tt_eppc_renewable

tt_eppc_renewable = tt_eppc_renewable[["State", "Electricity_Production_MWh"]]

# group dataframe by State
tt_eppc_renewable_grouped = tt_eppc_renewable.groupby(["State"], as_index=False).sum()
tt_eppc_renewable_grouped

# format DF
tt_eppc_renewable_grouped.rename(columns={'Electricity_Production_MWh': 'Renewable_Electricity_Production_MWh'}, inplace=True)
tt_eppc_renewable_grouped

,State,Renewable_Electricity_Production_MWh
0,ACT,"034,7124,19120,21031,140"
1,NSW,"159,867113,5919,32934233,367785,12112,029018,8..."
2,NT,"7,9803,3611,169"
3,QLD,"163,293209,77315,3612,508201,752188,998560,798..."
4,SA,"264,15317,6116,60217,429347,539439,6078,7741,6..."
5,TAS,"308,54090,608177,896297,797219,1447,016397,232..."
6,VIC,"282,916649,8962,01187,35642,576111,428733,8531..."
7,WA,"30,125472,73248,207245,541499,33021,49932,5996..."


In [149]:
tt_eppc = tt_transformed[["State", "Electricity_Production_MWh", "Renewable"]]
# tt_eppc

tt_eppc_non_renewable = tt_eppc.loc[tt_eppc["Renewable"] == False]
# tt_eppc_non_renewable

tt_eppc_non_renewable = tt_eppc_non_renewable[["State", "Electricity_Production_MWh"]]

# group dataframe by State
tt_eppc_non_renewable_grouped = tt_eppc_non_renewable.groupby(["State"], as_index=False).sum()
tt_eppc_non_renewable_grouped

# format DF
tt_eppc_non_renewable_grouped.rename(columns={'Electricity_Production_MWh': 'Non_Renewable_Electricity_Production_MWh'}, inplace=True)
tt_eppc_non_renewable_grouped

,State,Non_Renewable_Electricity_Production_MWh
0,NSW,"14,310,47607,307,116249,72992,10148,276163,269..."
1,NT,"328,429212,6851,5572394,1719139801,1089355,979..."
2,QLD,"112,8821,912,382409,8923,143,6135,057,2135,119..."
3,SA,"386,5551,668,98044,99704,80014,83219,5152,542,..."
4,VIC,"16,397,733127,914294,7128,902,31011,11711,5391..."
5,WA,"34,5111,594,393142,522747,5141,613,6101,689,84..."


In [150]:
eppc_merged = pd.merge(tt_eppc_renewable_grouped, tt_eppc_non_renewable_grouped, how="left", on=["State", "State"])
eppc_merged

,State,Renewable_Electricity_Production_MWh,Non_Renewable_Electricity_Production_MWh
0,ACT,"034,7124,19120,21031,140",NaN
1,NSW,"159,867113,5919,32934233,367785,12112,029018,8...","14,310,47607,307,116249,72992,10148,276163,269..."
2,NT,"7,9803,3611,169","328,429212,6851,5572394,1719139801,1089355,979..."
3,QLD,"163,293209,77315,3612,508201,752188,998560,798...","112,8821,912,382409,8923,143,6135,057,2135,119..."
4,SA,"264,15317,6116,60217,429347,539439,6078,7741,6...","386,5551,668,98044,99704,80014,83219,5152,542,..."
5,TAS,"308,54090,608177,896297,797219,1447,016397,232...",NaN
6,VIC,"282,916649,8962,01187,35642,576111,428733,8531...","16,397,733127,914294,7128,902,31011,11711,5391..."
7,WA,"30,125472,73248,207245,541499,33021,49932,5996...","34,5111,594,393142,522747,5141,613,6101,689,84..."


In [151]:
eppc_merged = pd.merge(eppc_merged, population_summary, how="left", on=["State", "State"])
eppc_merged

,State,Renewable_Electricity_Production_MWh,Non_Renewable_Electricity_Production_MWh,Population_Total
0,ACT,"034,7124,19120,21031,140",NaN,454499
1,NSW,"159,867113,5919,32934233,367785,12112,029018,8...","14,310,47607,307,116249,72992,10148,276163,269...",8072163
2,NT,"7,9803,3611,169","328,429212,6851,5572394,1719139801,1089355,979...",232605
3,QLD,"163,293209,77315,3612,508201,752188,998560,798...","112,8821,912,382409,8923,143,6135,057,2135,119...",5156138
4,SA,"264,15317,6116,60217,429347,539439,6078,7741,6...","386,5551,668,98044,99704,80014,83219,5152,542,...",1781516
5,TAS,"308,54090,608177,896297,797219,1447,016397,232...",NaN,557571
6,VIC,"282,916649,8962,01187,35642,576111,428733,8531...","16,397,733127,914294,7128,902,31011,11711,5391...",6503491
7,WA,"30,125472,73248,207245,541499,33021,49932,5996...","34,5111,594,393142,522747,5141,613,6101,689,84...",2660026


In [152]:
eppc_merged.replace(',','', regex=True, inplace=True)
eppc_merged = eppc_merged.fillna(0)
eppc_merged

,State,Renewable_Electricity_Production_MWh,Non_Renewable_Electricity_Production_MWh,Population_Total
0,ACT,03471241912021031140,0,454499
1,NSW,1598671135919329342333677851211202901883550511...,1431047607307116249729921014827616326967701198...,8072163
2,NT,798033611169,3284292126851557239417191398011089355979387131...,232605
3,QLD,1632932097731536125082017521889985607981879381...,1128821912382409892314361350572135119046284714...,5156138
4,SA,2641531761166021742934753943960787741613207506...,3865551668980449970480014832195152542096749673...,1781516
5,TAS,3085409060817789629779721914470163972321467771...,0,557571
6,VIC,2829166498962011873564257611142873385310744217...,1639773312791429471289023101111711539115402737...,6503491
7,WA,3012547273248207245541499330214993259969069318...,3451115943931425227475141613610168984120421880...,2660026


In [154]:
eppc_merged["Renewable_Electricity_Production_MWh"] = pd.to_numeric(eppc_merged["Renewable_Electricity_Production_MWh"])

ValueError: Unable to parse string "15986711359193293423336778512112029018835505113866221940051649745069469769710694544869029120141162532101802350182922851484828968913218583111131345129810035333346355061143467620020606001447251851118972484009804314730817543428785848169311125849177813866265205012413140842210050476688213466220078829172533608783521565489737538" at position 1

In [153]:
# create a column for "Per Student Budget" in the school_summary_merge dataframe
eppc_merged['Renewable_MWh_Per_Capita'] = eppc_merged['Renewable_Electricity_Production_MWh'] / eppc_merged['Population_Total'] 
eppc_merged['Non_Renewable_MWh_Per_Capita'] = eppc_merged['Non_Renewable_Electricity_Production_MWh'] / eppc_merged['Population_Total'] 

eppc_merged

TypeError: unsupported operand type(s) for /: 'str' and 'int'